In [15]:
import pandas as pd
import contractions

In [16]:
train = pd.read_json('../data/raw/dialogsum/dialogsum.train.jsonl', lines = True)
dev = pd.read_json('../data/raw/dialogsum/dialogsum.dev.jsonl', lines= True)

In [17]:
train_corpus = list(train['dialogue'])
dev_corpus = list(dev['dialogue'])

In [18]:
corpus = []
corpus.extend(train_corpus)
corpus.extend(dev_corpus)

In [19]:
def clean(text):
    return contractions.fix(text.lower().replace('\n', '\t'))

corpus = list(map(clean, corpus))

In [20]:
corpus = '\n'.join(corpus)

In [21]:
with open('../data/processed/corpus', 'w') as f:
    f.write(corpus)